In [1]:
!pip install --upgrade gspread

In [2]:
# Insere as credenciais para abrir o datagrame
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [3]:
import pandas as pd
import numpy as np

%precision %.2f
pd.options.display.float_format = '{:,.2f}'.format

def stringSearchColumn_DataFrame(df, colName, regex):
    newdf = pd.DataFrame()
    for idx, record in df[colName].iteritems():

        if re.search(regex, record):
            newdf = concat([df[df[colName] == record], newdf], ignore_index=True)

    return newdf

In [4]:
# Abre a planilha do google
#sh = gc.open('CargasSetores') # planilha com todas as cargas instaladas
sh = gc.open('Cargas2022') # planilha de cargas instaladas até 2022
page = sh.sheet1
rows = page.get_all_records()
# print(rows)

In [5]:
cargasSetores=pd.DataFrame.from_records(rows)
cargasSetores.head()

,setor,painel,tensao,tag,instalacao,remocao,marca,descricao,modelo,cv,kw,fp,amperes,rpm,kva,acionamento
0,industria,,380,,2018,,weg,MOINHO 1,280S/M,125.00,90.00,0.90,164.00,3575,107.94,soft-starter
1,industria,,380,,2018,,weg,MOINHO 2,280S/M,125.00,90.00,0.90,164.00,3575,107.94,soft-starter
2,industria,,380,,2018,,weg,ALIMENTADOR 1,71,0.50,0.40,0.80,1.10,1670,0.70,inversorES
3,industria,,380,,2018,,weg,ALIMENTADOR 2,71,0.50,0.40,0.80,1.10,1670,0.70,inversorES
4,industria,,380,,2018,,Metalplan,COMPRESSOR 1,TP 50hp,50.00,37.30,0.80,70.30,3560,46.27,ESTRELA /TRIANGUL


## INFORMAÇÕES DO BANCO DE DADOS

In [6]:
df = cargasSetores

print('Os setores unicos da usina são', df['setor'].unique())
print("Temos num total de " + str(len(df['setor'].unique())) + ' setores')

Os setores unicos da usina são ['industria' 'fermentacao' 'destilaria' 'secador' '' 'evaporacao'
 'caldeira02']
Temos num total de 7 setores


In [7]:
df[df['setor'] == "industria"].head()

,setor,painel,tensao,tag,instalacao,remocao,marca,descricao,modelo,cv,kw,fp,amperes,rpm,kva,acionamento
0,industria,,380,,2018,,weg,MOINHO 1,280S/M,125.00,90.00,0.90,164.00,3575,107.94,soft-starter
1,industria,,380,,2018,,weg,MOINHO 2,280S/M,125.00,90.00,0.90,164.00,3575,107.94,soft-starter
2,industria,,380,,2018,,weg,ALIMENTADOR 1,71,0.50,0.40,0.80,1.10,1670,0.70,inversorES
3,industria,,380,,2018,,weg,ALIMENTADOR 2,71,0.50,0.40,0.80,1.10,1670,0.70,inversorES
4,industria,,380,,2018,,Metalplan,COMPRESSOR 1,TP 50hp,50.00,37.30,0.80,70.30,3560,46.27,ESTRELA /TRIANGUL


### DESCRIÇÃO GERAL

In [8]:
cargasSetores.describe()

,tensao,cv,kw,fp,amperes,rpm,kva
count,215.00,215.00,215.00,215.00,215.00,215.00,215.00
mean,386.05,18.66,13.89,0.86,25.80,"1,663.61",17.34
std,46.23,35.67,26.41,0.05,45.36,921.74,31.67
min,220.00,0.30,0.20,0.70,0.30,0.00,0.22
25%,380.00,3.00,2.20,0.80,3.50,"1,725.00",2.64
50%,380.00,7.50,5.50,0.90,10.00,"1,760.00",6.57
75%,380.00,17.50,13.05,0.90,26.95,"1,780.00",17.45
max,440.00,300.00,223.70,0.90,346.80,"3,600.00",264.31


## Separando as cargas por setor

In [9]:
industria = df[df['setor'] == "industria"]
fermentacao = df[df['setor'] == "fermentacao"]
destilaria = df[df['setor'] == "destilaria"]
secador = df[df['setor'] == "secador"]
evaporacao = df[df['setor'] == "evaporacao"]
caldeira02 = df[df['setor'] == "caldeira02"]

## Obtendo a carga total instalada por setor da usina

### Caldeira 02

In [10]:
kva_caldeira02 = caldeira02.kva.sum()
kw_caldeira02 = caldeira02.kw.sum()
fp_caldeira02 = kw_caldeira02/kva_caldeira02
In_caldeira02 = kva_caldeira02/(13.8*np.sqrt(3))
print('A corrente da caldeira II em 13.8kV é %.2f Amperes. Em 440V é de %.2f Amperes.' %(In_caldeira02, (In_caldeira02*13800/440)))
print('A potência da caldeira II é de P=%.2f kW e S=%.2f kVA com um fator de potência de %.2f.' %(kw_caldeira02, kva_caldeira02, fp_caldeira02))

A corrente da caldeira II em 13.8kV é 31.98 Amperes. Em 440V é de 1002.88 Amperes.
A potência da caldeira II é de P=641.70 kW e S=764.30 kVA com um fator de potência de 0.84.


### Fermentação

In [11]:
kva_fermentacao = fermentacao.kva.sum()
kw_fermentacao = fermentacao.kw.sum()
fp_fermentacao = kw_fermentacao/kva_fermentacao
In_fermentacao = kva_fermentacao/(13.8*np.sqrt(3))
print('A corrente da fermentação em 13.8kV é %.2f Amperes. Em 380V é de %.2f Amperes.' %(In_fermentacao, (In_fermentacao*13800/380)))
print('A potência da fermentação é de P=%.2f kW e S=%.2f kVA com um fator de potência de %.2f.' %(kw_fermentacao, kva_fermentacao, fp_fermentacao))

A corrente da fermentação em 13.8kV é 11.75 Amperes. Em 380V é de 426.74 Amperes.
A potência da fermentação é de P=223.00 kW e S=280.87 kVA com um fator de potência de 0.79.


### Idustria

In [12]:
kva_industria = industria.kva.sum()
kw_industria = industria.kw.sum()
fp_industria = kw_industria/kva_industria
In_industria = kva_industria/(13.8*np.sqrt(3))
print('A corrente da industria em 13.8kV é %.2f Amperes. Em 380V é de %.2f Amperes.' %(In_industria, (In_industria*13800/380)))
print('A potência da industria é de P=%.2f kW e S=%.2f kVA com um fator de potência de %.2f.' %(kw_industria, kva_industria, fp_industria))

A corrente da industria em 13.8kV é 33.75 Amperes. Em 380V é de 1225.84 Amperes.
A potência da industria é de P=633.50 kW e S=806.82 kVA com um fator de potência de 0.79.


### Secador

In [13]:
kva_secador = secador.kva.sum()
kw_secador = secador.kw.sum()
fp_secador = kw_secador/kva_secador
In_secador = kva_secador/(13.8*np.sqrt(3))
print('A corrente da secador em 13.8kV é %.2f Amperes. Em 380V é de %.2f Amperes.' %(In_secador, (In_secador*13800/380)))
print('A potência da secador é de P=%.2f kW e S=%.2f kVA com um fator de potência de %.2f.' %(kw_secador, kva_secador, fp_secador))

A corrente da secador em 13.8kV é 22.74 Amperes. Em 380V é de 825.82 Amperes.
A potência da secador é de P=435.70 kW e S=543.54 kVA com um fator de potência de 0.80.


### Destilaria + Evaporação


In [14]:
kva_destilaria = destilaria.kva.sum()
kw_destilaria = destilaria.kw.sum()
fp_destilaria = kw_destilaria/kva_destilaria
In_destilaria = kva_destilaria/(13.8*np.sqrt(3))
print('A corrente da destilaria em 13.8kV é %.2f Amperes. Em 380V é de %.2f Amperes.' %(In_destilaria, (In_destilaria*13800/380)))
print('A potência da destilaria é de P=%.2f kW e S=%.2f kVA com um fator de potência de %.2f.' %(kw_destilaria, kva_destilaria, fp_destilaria))

A corrente da destilaria em 13.8kV é 15.77 Amperes. Em 380V é de 572.56 Amperes.
A potência da destilaria é de P=307.40 kW e S=376.85 kVA com um fator de potência de 0.82.


In [15]:
kva_evaporacao = evaporacao.kva.sum()
kw_evaporacao = evaporacao.kw.sum()
fp_evaporacao = kw_evaporacao/kva_evaporacao
In_evaporacao = kva_evaporacao/(13.8*np.sqrt(3))
print('A corrente da evaporação em 13.8kV é %.2f Amperes. Em 380V é de %.2f Amperes.' %(In_evaporacao, (In_evaporacao*13800/380)))
print('A potência da evaporação é de P=%.2f kW e S=%.2f kVA com um fator de potência de %.2f.' %(kw_evaporacao, kva_evaporacao, fp_evaporacao))

A corrente da evaporação em 13.8kV é 37.17 Amperes. Em 380V é de 1349.75 Amperes.
A potência da evaporação é de P=689.80 kW e S=888.38 kVA com um fator de potência de 0.78.


In [16]:
print('A potência total dos setores da evaporação + destilaria é de %.2f kW.' %(kw_destilaria+kw_evaporacao))
print('A potência aparente dos setores da evaporação + destilaria é de %.2f kVA.' %(kva_destilaria+kva_evaporacao))
print('O fator de potência da destilaria + evaporação %.2f.' %((kw_destilaria+kw_evaporacao)/(kva_destilaria+kva_evaporacao)))

A potência total dos setores da evaporação + destilaria é de 997.20 kW.
A potência aparente dos setores da evaporação + destilaria é de 1265.23 kVA.
O fator de potência da destilaria + evaporação 0.79.


## VERIFICANDO POTÊNCIA  TOTAL INSTALADA (PI)

In [17]:
# Somando as potências e obtendo o fator de potencia da usina
kva_total = cargasSetores.kva.sum()
kw_total = cargasSetores.kw.sum()
fp_total = kw_total/kva_total
fp_total

0.80

## Tabela com as cargas dos setores


A potência total intalada é dada pela soma em kw de todas as cargas.
PI = soma(kw)

In [18]:
print("A potência total instalação é de %.2f kW" %(kw_total))
print('A potência aparente total da usina é de %.2f kVA com um fator de potência de %.2f.' %(kva_total, fp_total))

A potência total instalação é de 2987.00 kW
A potência aparente total da usina é de 3727.89 kVA com um fator de potência de 0.80.


### Corrente total em 13,8kV

In [19]:
In_total = kva_total/(13.8*np.sqrt(3))
print('A corrente total em 13.8kV é %.2f Amperes.' %(In_total))

A corrente total em 13.8kV é 155.96 Amperes.


# DIMENSIONAMENTO DO TRAFO DA SUB DE ENTRADA

Para dimensionamento do tranformador da subestação, é necessário levar em consideração alguns parâmetros além da potência instalada.

### Fator de demanda (FD)

É muito raro em uma instalação elétrica utilizarmos todos os equipamentos instalados. Supondo que utilizamos 80% de uma carga instalada de 100KW. Nosso fator de demanda de 80% vai resultar em uma potência consumida de 80kW.

### Fator de crescimento de carga (PAC)

O fator de crescimento de carga deve ser considerado em todo projeto para liberar uma margem de aumento de carga. Vamos considerar 30% de aumento da carga instalada em 5 anos.

### Fator de potência (FP)

A legislação brasileira exige que o fator de potência não seja inferior a 0,92. 

### Calculo do trafo

Considerando esses fatores a potência do trafo pode ser dada por:

$ S_{Trafo} =  \frac{PI×FD×PAC}{FP} (KVA)$

In [20]:
FD = 0.8 # fator de demanda
FP = 0.92 # fator de potência 
PAC = 1.3 # crescimento de carga
PI = kw_total

S_trafo = FD*PAC*PI/FP

print("A potência aparente total do \n tranformador deve ser de %.2f KVA" %(S_trafo))

A potência aparente total do 
 tranformador deve ser de 3376.61 KVA
